In [1]:
# Para testes
import numpy as np
import matplotlib.pyplot as plt

pi = np.pi

In [2]:
from typing import Callable

# Cinemática Inversa

O problema da cinemática inversa consiste em encontrar ângulos de juntas robóticas que correspondem à certa posição do braço robótico no espaço. Ou seja, encontrar valores de $\theta$ que satisfaçam $f(\theta)=x$, $f(\theta_1,\theta_2)=(x,y)$ e $f(\theta_1,\theta_2,\theta_3)=(x,y,z)$.

## Método de Newton

In [3]:
def nr(f: Callable[[float], float], df: Callable[[float], float], x: float, p: float) -> float:
    # p = chute inicial
    def g(w):
        return f(w) - x
    xl = p
    # critérios de parada
    N, e_stop = 50, 1e-8
    iter, e = 0, 1
    while iter < N-1 and e > e_stop:
        xn = xl - g(xl)/df(xl)
        e = xn
        xl = xn
        iter += 1

    return xn

## Caso para 1 dimensão

O caso para 1 dimensão se resume em encontrar um ângulo referente a uma posição em um círculo

In [4]:
R = 3

def f_1d(theta: float) -> float:
    return theta*R

def d_f_1d(theta: float) -> float:
    return R

In [5]:
theta_nr = nr(f_1d, d_f_1d, 2, 0.7)
print(f"Valor de theta encontrado para x = 2: {theta_nr}")
print(f"Valor de x: {f_1d(theta_nr)}")

Valor de theta encontrado para x = 2: 0.6666666666666667
Valor de x: 2.0


## Caso para 2 dimensões

Nesse caso se tem um um sistema com dois braços, braço 1 e braço 2. A posição desejada se dará pela posição da ponta não articulada do braço 2. Considerando os ângulos $\theta_1$ (calculado entre a reta do braço 1 e a reta $x=0$) e $\theta_2$ (calculado entre a reta do braço 2 e a paralela à reta $x=0$), a posição desejada se dá por:

$(x,y)=\Big(f_1(\theta_1,\theta_2), f_2(\theta_1,\theta_2)\Big)=\Big(R_1\cos(\theta_1)+R_2\cos(\theta_2), R_1\sin(\theta_1)+R_2\sin(\theta_2)\Big)$

$\frac{df}{d\theta}=
\begin{bmatrix}
\frac{\partial f_1}{\partial\theta_1}&\frac{\partial f_1}{\partial\theta_2}\\
\frac{\partial f_2}{\partial\theta_1}&\frac{\partial f_2}{\partial\theta_2}
\end{bmatrix}=
\begin{bmatrix}
-R_1\sin(\theta_1)&-R_2\sin(\theta_2)\\
R_1\cos(\theta_1)&R_2\cos(\theta_2)
\end{bmatrix}$

### Método da descida do gradiente

Nesse caso, deve-se aplicar o método para $f_1$ e $f_2$.
As funções a serem minimizadas serão
$\Big(f_1(\theta_1, \theta_2) - x\Big)^2$ e $\Big(f_2(\theta_1, \theta_2) - y\Big)^2$.

$\nabla f_1 =
\Big( 2\cdot\big[f_1(\theta_1, \theta_2) - x\big]\frac{\partial f_1}{\partial\theta_1},
2\cdot\big[f_1(\theta_1, \theta_2) - x\big]\frac{\partial f_1}{\partial\theta_2} \Big)$

$\nabla f_2 =
\Big( 2\cdot\big[f_2(\theta_1, \theta_2) - y\big]\frac{\partial f_2}{\partial\theta_1},
2\cdot\big[f_2(\theta_1, \theta_2) - y\big]\frac{\partial f_2}{\partial\theta_2} \Big)$

**OBS:** Assim não funcionaria, pois seria possível encontrar valores diferentes para $\theta_1$ e $\theta_2$ em $f_1$ e $f_2$.

In [50]:
R1, R2 = 3, 2.5

# f_2d: (R1*np.cos(theta1)+R2*np.cos(theta2), R1*np.sin(theta1)+R2*np.sin(theta2))
# grad_f_2d: ( (-R1*np.sin(theta1), -R2*np.sin(theta2)), (R1*np.cos(theta1), R2*np.cos(theta2)) )

def f(theta1: float, theta2: float, func: Callable) -> float:
    return R1*func(theta1) + R2*func(theta2)

def f_2d_x(theta1: float, theta2: float, x: float) -> float:
    return (f(theta1, theta2, np.cos) - x)**2

def f_2d_y(theta1: float, theta2: float, y: float) -> float:
    return (f(theta1, theta2, np.sin) - y)**2

def grad_f_2d_x(theta1: float, theta2: float, x: float) -> tuple:
    w = (f(theta1, theta2, np.cos)-x)
    return ((2*w*(-R1*np.sin(theta1))),
            (2*w*(-R2*np.sin(theta2))))

def grad_f_2d_y(theta1: float, theta2: float, y: float) -> tuple:
    w = (f(theta1, theta2, np.sin)-y)
    return (2*w*R1*np.cos(theta1),
            2*w*R2*np.cos(theta2))

In [69]:
def descida_grad(f: Callable[[float, float, float], float],
                 df: Callable[[float, float, float], tuple],
                 k: float, x0y0: tuple = (0.5, 0.5)):
    # passo fixo
    h = 0.7
    # x0y0 = chute inicial
    xo, yo = x0y0[0], x0y0[1]
    # critérios de parada
    N, tol = 50, 1e-8
    iter, grad = 0, (tol + 1, tol + 1)
    while iter < N and grad[0] > tol and grad[1] > tol:
        grad = df(xo, yo, k)
        xnyn = (
            (xo - grad[0]*h),
            (yo - grad[1]*h)
        )
        xo, yo = xnyn[0], xnyn[1]
        iter += 1

    return xnyn

In [70]:
# x
x = 2
theta1, theta2 = descida_grad(f_2d_x, grad_f_2d_x, x, (0.8, 0.8))
print(f"Valores de theta encontrados para x = {x}: {theta1, theta2}")
print(f"Valor de x: {f(theta1, theta2, np.cos)}")
print(f_2d_x(theta1, theta2, 0.5))

Valores de theta encontrados para x = 2: (6.319283951573393, 5.399403292977827)
Valor de x: 4.583624666511952
16.67599041694485
